In [1]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "2B3nARpwz82J49u.root",
  password = "tpc7Iw6jTcsoavz2",
  database = "TextToVector",
  ssl_ca = "isrgrootx1 (2).pem",
  ssl_verify_cert = True,
  ssl_verify_identity = True
)

In [2]:
print(connection)

In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

c:\Users\User\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
local_path = 'scammer-agent.pdf'
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f'PDF loaded successfully: {local_path}')
else:
    print('Upload a PDF file')

PDF loaded successfully: scammer-agent.pdf


In [8]:
data

[Document(metadata={'source': 'scammer-agent.pdf'}, page_content='4 2 0 2\n\nt c O 1 2\n\n] I\n\nA . s c [\n\n1 v 0 5 6 5 1 . 0 1 4 2 : v i X r a\n\nVoice-Enabled AI Agents can Perform Common Scams\n\nRichard Fang UIUC\n\nDylan Bowman UIUC\n\nDaniel Kang UIUC\n\nAbstract\n\nRecent advances in multi-modal, highly ca- pable LLMs have enabled voice-enabled AI agents. These agents are enabling new applica- tions, such as voice-enabled autonomous cus- tomer service. However, with all AI capabili- ties, these new capabilities have the potential for dual use.\n\nIn this work, we show that voice-enabled AI agents can perform the actions necessary to perform common scams. To do so, we select a list of common scams collected by the govern- ment and construct voice-enabled agents with directions to perform these scams. We con- duct experiments on our voice-enabled agents and show that they can indeed perform the ac- tions necessary to autonomously perform such scams. Our results raise questions a

## Clean Text

In [14]:
import re
import unicodedata
def clean_text(data):
    text = re.sub(r'^\s*\d+\s*$', '', data, flags=re.MULTILINE)
    text = re.sub(r'^[^\w\s]{1,3}$', '', text, flags=re.MULTILINE)
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    text = re.sub(r'\n{2,}', '\n\n', text)
    text = re.sub(r'-\s+', '', text)
    text = re.sub(r'(?<=\b\w)\s(?=\w\b)', '', text)
    text = unicodedata.normalize("NFKC", text)
    return text.strip()

In [15]:
data_clean = []
for doc in data:
    raw_text = doc.page_content
    text = clean_text(raw_text)
    data_clean.append(text)

In [16]:
print(data_clean)

['4202\n\ntcO12\n\n] I\n\nA . sc [\n\n1v05651 . 0142 : viXra\n\nVoice-Enabled AI Agents can Perform Common Scams\n\nRichard Fang UIUC\n\nDylan Bowman UIUC\n\nDaniel Kang UIUC\n\nAbstract\n\nRecent advances in multi-modal, highly capable LLMs have enabled voice-enabled AI agents. These agents are enabling new applications, such as voice-enabled autonomous customer service. However, with all AI capabilities, these new capabilities have the potential for dual use.\n\nIn this work, we show that voice-enabled AI agents can perform the actions necessary to perform common scams. To do so, we select a list of common scams collected by the government and construct voice-enabled agents with directions to perform these scams. We conduct experiments on our voice-enabled agents and show that they can indeed perform the actions necessary to autonomously perform such scams. Our results raise questions around the widespread deployment of voice-enabled AI agents.\n\nIntroduction\n\nAI capabilities have

## Chunking


In [ ]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import nltk


